### Data loading overview
This bookm serves to load the data and get a understanding for it. This book will only consider data from the following csv files: train, test and structures


In [3]:
import os

input_folder = './input'

os.listdir(input_folder)

['dipole_moments.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'sample_submission.csv',
 'scalar_coupling_contributions.csv',
 'structures.csv',
 'structures.zip',
 'test.csv',
 'train.csv']

In [5]:
import pandas as pd

train = pd.read_csv(f'{input_folder}/train.csv')
test = pd.read_csv(f'{input_folder}/test.csv')
structures = pd.read_csv(f'{input_folder}/structures.csv')

In [8]:
f'Train shape: {train.shape}'

'Train shape: (4658147, 6)'

In [13]:
train.head(5)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [9]:
f'Test shape: {test.shape}'

'Test shape: (2505542, 5)'

In [12]:
test.head(5)

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [10]:
f'Structures shape: {structures.shape}'

'Structures shape: (2358657, 6)'

In [14]:
structures.head(5)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [118]:
print(f"There are {train['molecule_name'].nunique()} distinct molecules in train data.")
print(f"There are {test['molecule_name'].nunique()} distinct molecules in test data.")
print(f"Largest molecule contains upto {max(train['atom_index_0'].nunique(), train['atom_index_1'].nunique())} atoms.")
print(f"There are {train['type'].nunique()} unique coupling types: {train['type'].unique()}")
print(f"There are {structures['atom'].nunique()} unique atom types: {structures['atom'].unique()}")
      

There are 85003 distinct molecules in train data.
There are 45772 distinct molecules in test data.
Largest molecule contains upto 29 atoms.
There are 8 unique coupling types: ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']
There are 5 unique atom types: ['C' 'H' 'N' 'O' 'F']


In [36]:
train_molecule_set = set(train['molecule_name'].unique())
test_molecule_set = set(test['molecule_name'].unique())
struct_molecule_set = set(structures['molecule_name'].unique())

molecule_overlap_test_train = train_molecule_set.intersection(test_molecule_set)
print('Molecule overlap bewteen training and test data sets: {}'.format(len(molecule_overlap_test_train)))

molecule_overlap_train_in_struct = train_molecule_set.intersection(struct_molecule_set)
print('Train molecules described in structures file: {}%'.format(len(molecule_overlap_train_in_struct)*100/len(train_molecule_set)))

molecule_overlap_test_in_struct = test_molecule_set.intersection(struct_molecule_set)
print('Test molecules described in structures file: {}%'.format(len(molecule_overlap_test_in_struct)*100/len(test_molecule_set)))

Molecule overlap bewteen training and test data sets: 0
Train molecules described in structures file: 100.0%
Test molecules described in structures file: 100.0%


#### Any missing values?

In [109]:
import numpy as np

pd.set_option('use_inf_as_na', True)

# replace field that's entirely space (or empty) with NaN
train=train.replace(r'^\s*$', np.nan, regex=True)
test=test.replace(r'^\s*$', np.nan, regex=True)
structures=structures.replace(r'^\s*$', np.nan, regex=True)

In [110]:
train.isna().any()

id                          False
molecule_name               False
atom_index_0                False
atom_index_1                False
type                        False
scalar_coupling_constant    False
dtype: bool

In [111]:
test.isna().any()

id               False
molecule_name    False
atom_index_0     False
atom_index_1     False
type             False
dtype: bool

In [112]:
structures.isna().any()

molecule_name    False
atom_index       False
atom             False
x                False
y                False
z                False
dtype: bool

#### Merging data sources into one training data set and one test data set 

Code for joining data is from this kernel: https://www.kaggle.com/seriousran/just-speed-up-calculate-distance-from-benchmark

In [113]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [114]:
train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

In [116]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [115]:
test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [117]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


#### Data splitting

In [148]:
train_X = train.iloc[:,2:]
train_X = train_X.drop('scalar_coupling_constant', 1)
test_X = test.iloc[:,2:]
train_y= train['scalar_coupling_constant']

In [153]:
print(train_X.dtypes)

atom_index_0      int64
atom_index_1      int64
type             object
atom_0           object
x_0             float64
y_0             float64
z_0             float64
atom_1           object
x_1             float64
y_1             float64
z_1             float64
dtype: object


In [154]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, random_state = 13, test_size = 0.25)


### The model

In [155]:
from catboost import CatBoostRegressor

In [156]:
categorical_features_indices = np.where(train_X.dtypes != np.float)[0]

In [157]:
basic_model = CatBoostRegressor(task_type = "GPU", devices='0', gpu_ram_part=0.95, gpu_cat_features_storage='GpuRam',  iterations=12000, random_seed=42, loss_function='MAE', custom_metric=['MAE'], eval_metric='MAE', grow_policy='SymmetricTree', border_count=255, random_strength=2, learning_rate=0.03, l2_leaf_reg=6, depth=3)


In [158]:
basic_model.fit(X_train, y_train, 
          cat_features=categorical_features_indices, 
          eval_set=(X_valid, y_valid),
          logging_level='Verbose',
          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 18.3688723	test: 18.4076624	best: 18.4076624 (0)	total: 107ms	remaining: 21m 23s
1:	learn: 18.3594322	test: 18.3982235	best: 18.3982235 (1)	total: 221ms	remaining: 22m 4s
2:	learn: 18.3500849	test: 18.3888636	best: 18.3888636 (2)	total: 331ms	remaining: 22m 4s
3:	learn: 18.3407581	test: 18.3795328	best: 18.3795328 (3)	total: 446ms	remaining: 22m 17s
4:	learn: 18.3314966	test: 18.3702708	best: 18.3702708 (4)	total: 569ms	remaining: 22m 44s
5:	learn: 18.3222970	test: 18.3610499	best: 18.3610499 (5)	total: 648ms	remaining: 21m 36s
6:	learn: 18.3131294	test: 18.3518892	best: 18.3518892 (6)	total: 789ms	remaining: 22m 31s
7:	learn: 18.3040637	test: 18.3427852	best: 18.3427852 (7)	total: 882ms	remaining: 22m 1s
8:	learn: 18.2950232	test: 18.3337275	best: 18.3337275 (8)	total: 960ms	remaining: 21m 19s
9:	learn: 18.2860010	test: 18.3247299	best: 18.3247299 (9)	total: 1.04s	remaining: 20m 47s
10:	learn: 18.2770636	test: 18.3157615	best: 18.3157615 (10)	total: 1.18s	remaining: 21m 26s


89:	learn: 17.6970194	test: 17.7352012	best: 17.7352012 (89)	total: 11.9s	remaining: 26m 17s
90:	learn: 17.6908985	test: 17.7290597	best: 17.7290597 (90)	total: 12.1s	remaining: 26m 17s
91:	learn: 17.6847834	test: 17.7229371	best: 17.7229371 (91)	total: 12.2s	remaining: 26m 17s
92:	learn: 17.6786888	test: 17.7168540	best: 17.7168540 (92)	total: 12.4s	remaining: 26m 23s
93:	learn: 17.6726526	test: 17.7107898	best: 17.7107898 (93)	total: 12.5s	remaining: 26m 22s
94:	learn: 17.6665707	test: 17.7047359	best: 17.7047359 (94)	total: 12.6s	remaining: 26m 22s
95:	learn: 17.6605838	test: 17.6987077	best: 17.6987077 (95)	total: 12.7s	remaining: 26m 20s
96:	learn: 17.6544812	test: 17.6926641	best: 17.6926641 (96)	total: 12.9s	remaining: 26m 26s
97:	learn: 17.6485664	test: 17.6866806	best: 17.6866806 (97)	total: 13.1s	remaining: 26m 30s
98:	learn: 17.6425829	test: 17.6806989	best: 17.6806989 (98)	total: 13.2s	remaining: 26m 29s
99:	learn: 17.6365628	test: 17.6746449	best: 17.6746449 (99)	total: 13

177:	learn: 17.2264139	test: 17.2642054	best: 17.2642054 (177)	total: 23.9s	remaining: 26m 26s
178:	learn: 17.2217288	test: 17.2594928	best: 17.2594928 (178)	total: 24s	remaining: 26m 22s
179:	learn: 17.2170792	test: 17.2548369	best: 17.2548369 (179)	total: 24.1s	remaining: 26m 25s
180:	learn: 17.2122441	test: 17.2499972	best: 17.2499972 (180)	total: 24.3s	remaining: 26m 24s
181:	learn: 17.2075773	test: 17.2453550	best: 17.2453550 (181)	total: 24.3s	remaining: 26m 20s
182:	learn: 17.2029746	test: 17.2406974	best: 17.2406974 (182)	total: 24.5s	remaining: 26m 20s
183:	learn: 17.1983146	test: 17.2360655	best: 17.2360655 (183)	total: 24.6s	remaining: 26m 16s
184:	learn: 17.1937177	test: 17.2314508	best: 17.2314508 (184)	total: 24.7s	remaining: 26m 17s
185:	learn: 17.1890921	test: 17.2268326	best: 17.2268326 (185)	total: 24.8s	remaining: 26m 17s
186:	learn: 17.1843371	test: 17.2220805	best: 17.2220805 (186)	total: 25s	remaining: 26m 21s
187:	learn: 17.1795993	test: 17.2173336	best: 17.21733

264:	learn: 16.8508586	test: 16.8880869	best: 16.8880869 (264)	total: 35.2s	remaining: 25m 57s
265:	learn: 16.8469108	test: 16.8841385	best: 16.8841385 (265)	total: 35.3s	remaining: 25m 57s
266:	learn: 16.8429642	test: 16.8801987	best: 16.8801987 (266)	total: 35.5s	remaining: 25m 58s
267:	learn: 16.8390267	test: 16.8762470	best: 16.8762470 (267)	total: 35.6s	remaining: 25m 59s
268:	learn: 16.8351052	test: 16.8723072	best: 16.8723072 (268)	total: 35.8s	remaining: 25m 59s
269:	learn: 16.8311678	test: 16.8683674	best: 16.8683674 (269)	total: 35.9s	remaining: 26m
270:	learn: 16.8272486	test: 16.8644466	best: 16.8644466 (270)	total: 36.1s	remaining: 26m 1s
271:	learn: 16.8233432	test: 16.8605377	best: 16.8605377 (271)	total: 36.2s	remaining: 26m 1s
272:	learn: 16.8194584	test: 16.8566392	best: 16.8566392 (272)	total: 36.4s	remaining: 26m 2s
273:	learn: 16.8155885	test: 16.8527441	best: 16.8527441 (273)	total: 36.5s	remaining: 26m 3s
274:	learn: 16.8117037	test: 16.8488369	best: 16.8488369 (

...

11652:	learn: 3.7031706	test: 3.7010348	best: 3.7010348 (11652)	total: 23m 27s	remaining: 41.9s
11653:	learn: 3.7031643	test: 3.7010271	best: 3.7010271 (11653)	total: 23m 27s	remaining: 41.8s
11654:	learn: 3.7031569	test: 3.7010194	best: 3.7010194 (11654)	total: 23m 27s	remaining: 41.7s
11655:	learn: 3.7031538	test: 3.7010159	best: 3.7010159 (11655)	total: 23m 27s	remaining: 41.6s
11656:	learn: 3.7031314	test: 3.7009923	best: 3.7009923 (11656)	total: 23m 28s	remaining: 41.4s
11657:	learn: 3.7031251	test: 3.7009859	best: 3.7009859 (11657)	total: 23m 28s	remaining: 41.3s
11658:	learn: 3.7031163	test: 3.7009799	best: 3.7009799 (11658)	total: 23m 28s	remaining: 41.2s
11659:	learn: 3.7031014	test: 3.7009618	best: 3.7009618 (11659)	total: 23m 28s	remaining: 41.1s
11660:	learn: 3.7030934	test: 3.7009550	best: 3.7009550 (11660)	total: 23m 28s	remaining: 41s
11661:	learn: 3.7030839	test: 3.7009464	best: 3.7009464 (11661)	total: 23m 28s	remaining: 40.8s
11662:	learn: 3.7030770	test: 3.7009382	be

11738:	learn: 3.7022750	test: 3.7001422	best: 3.7001422 (11738)	total: 23m 39s	remaining: 31.6s
11739:	learn: 3.7022610	test: 3.7001250	best: 3.7001250 (11739)	total: 23m 39s	remaining: 31.4s
11740:	learn: 3.7022470	test: 3.7001113	best: 3.7001113 (11740)	total: 23m 39s	remaining: 31.3s
11741:	learn: 3.7022149	test: 3.7000769	best: 3.7000769 (11741)	total: 23m 39s	remaining: 31.2s
11742:	learn: 3.7022106	test: 3.7000731	best: 3.7000731 (11742)	total: 23m 39s	remaining: 31.1s
11743:	learn: 3.7022049	test: 3.7000679	best: 3.7000679 (11743)	total: 23m 39s	remaining: 30.9s
11744:	learn: 3.7021992	test: 3.7000611	best: 3.7000611 (11744)	total: 23m 40s	remaining: 30.8s
11745:	learn: 3.7021966	test: 3.7000580	best: 3.7000580 (11745)	total: 23m 40s	remaining: 30.7s
11746:	learn: 3.7021920	test: 3.7000542	best: 3.7000542 (11746)	total: 23m 40s	remaining: 30.6s
11747:	learn: 3.7021811	test: 3.7000447	best: 3.7000447 (11747)	total: 23m 40s	remaining: 30.5s
11748:	learn: 3.7021743	test: 3.7000400	

11825:	learn: 3.7013779	test: 3.6992298	best: 3.6992298 (11825)	total: 23m 50s	remaining: 21.1s
11826:	learn: 3.7013751	test: 3.6992260	best: 3.6992260 (11826)	total: 23m 50s	remaining: 20.9s
11827:	learn: 3.7013631	test: 3.6992144	best: 3.6992144 (11827)	total: 23m 51s	remaining: 20.8s
11828:	learn: 3.7013513	test: 3.6992006	best: 3.6992006 (11828)	total: 23m 51s	remaining: 20.7s
11829:	learn: 3.7013462	test: 3.6991933	best: 3.6991933 (11829)	total: 23m 51s	remaining: 20.6s
11830:	learn: 3.7013402	test: 3.6991903	best: 3.6991903 (11830)	total: 23m 51s	remaining: 20.4s
11831:	learn: 3.7013324	test: 3.6991826	best: 3.6991826 (11831)	total: 23m 51s	remaining: 20.3s
11832:	learn: 3.7013236	test: 3.6991723	best: 3.6991723 (11832)	total: 23m 51s	remaining: 20.2s
11833:	learn: 3.7013216	test: 3.6991689	best: 3.6991689 (11833)	total: 23m 51s	remaining: 20.1s
11834:	learn: 3.7013135	test: 3.6991633	best: 3.6991633 (11834)	total: 23m 51s	remaining: 20s
11835:	learn: 3.7013107	test: 3.6991598	be

11911:	learn: 3.7006071	test: 3.6984634	best: 3.6984634 (11911)	total: 24m 2s	remaining: 10.7s
11912:	learn: 3.7005745	test: 3.6984334	best: 3.6984334 (11912)	total: 24m 2s	remaining: 10.5s
11913:	learn: 3.7005570	test: 3.6984140	best: 3.6984140 (11913)	total: 24m 2s	remaining: 10.4s
11914:	learn: 3.7005444	test: 3.6984003	best: 3.6984003 (11914)	total: 24m 2s	remaining: 10.3s
11915:	learn: 3.7005161	test: 3.6983763	best: 3.6983763 (11915)	total: 24m 2s	remaining: 10.2s
11916:	learn: 3.7005106	test: 3.6983707	best: 3.6983707 (11916)	total: 24m 2s	remaining: 10.1s
11917:	learn: 3.7005043	test: 3.6983664	best: 3.6983664 (11917)	total: 24m 3s	remaining: 9.93s
11918:	learn: 3.7004989	test: 3.6983617	best: 3.6983617 (11918)	total: 24m 3s	remaining: 9.81s
11919:	learn: 3.7004923	test: 3.6983552	best: 3.6983552 (11919)	total: 24m 3s	remaining: 9.69s
11920:	learn: 3.7004837	test: 3.6983475	best: 3.6983475 (11920)	total: 24m 3s	remaining: 9.57s
11921:	learn: 3.7004797	test: 3.6983432	best: 3.69

11999:	learn: 3.6996808	test: 3.6975364	best: 3.6975364 (11999)	total: 24m 14s	remaining: 0us
bestTest = 3.697536446
bestIteration = 11999
Shrink model to first 12000 iterations.


In [159]:
y_pred = basic_model.predict(X_valid)
print(y_pred[:5])

[ 3.74088355 -2.10373514 90.50100132  3.77657046 -0.40517706]


In [160]:
# Competition metric from here https://www.kaggle.com/uberkinder/efficient-metric

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()


In [161]:
score = group_mean_log_mae(y_valid, y_pred, train['type'])
print(f'score {score}')

score 1.1405594098789402


Above score currently takes you to top 85%

### Let's submit it!
Utilty code for generating csv file of test predictions

In [162]:
predictions = basic_model.predict(test_X)

submission = pd.read_csv('./input/sample_submission.csv')
submission['scalar_coupling_constant'] = predictions
submission.to_csv('basic_CAT.csv', index=False)